In [3]:
import requests
import pandas as pd
import geohash2

In [2]:
#!pip install geohash2

  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=4d3caa5d701565f6120322f0a75eb1986faba296c2bfafb19a27d2de89deca10
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2d\0c\66\dcd768c9e7c26fc81ec59bfe24af9d780fc9dbdf7f90b0cf00
Successfully built geohash2


In [8]:
#1. 동이름 ==> 위도, 경도
#search? 에 header의 주소를 가져온다
#깨진 글은 https://meyerweb.com/eric/tools/dencoder/ 에서 decoding
addr = "성산구 상남동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(35.21626663208008, 128.6872100830078)

In [ ]:
#2. 위도, 경도 ==> geohash

In [10]:
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wy782'

In [12]:
#3. geohash ==> 매물 아이디
# items?deposit_gteq
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [15]:
items = response.json()['items']
len(items), items[:2]

(207,
 [{'lat': 35.20531103880102, 'lng': 128.7013631815903, 'item_id': 32598318},
  {'lat': 35.20485005103786, 'lng': 128.70139904185925, 'item_id': 32773496}])

In [18]:
ids = [items['item_id'] for items in items]
ids[:5]

[32598318, 32773496, 32483503, 32535077, 32535095]

In [19]:
#4. 매물 아이디 ==> 매물 정보
#list에 header url : https://apis.zigbang.com/v2/items/list
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain" : "zigbang",
    "withCoalition" : "true",
    "item_ids" : ids[:900] 
}

response = requests.post(url,params)
response

<Response [200]>

In [33]:
items = response.json()['items']
columns = ['item_id', 'sales_type','deposit','rent','size_m2','address1','manage_cost']
items[:2]
df = pd.DataFrame(items)
df[columns]

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32598318,월세,300,40,29.75,경상남도 창원시 성산구 가음동,0
1,32773496,월세,300,35,29.75,경상남도 창원시 성산구 가음동,2
2,32483503,월세,300,30,33.06,경상남도 창원시 성산구 가음동,1
3,32535077,월세,300,25,23.14,경상남도 창원시 성산구 남양동,1
4,32535095,월세,300,27,26.45,경상남도 창원시 성산구 남양동,1
...,...,...,...,...,...,...,...
202,32813383,월세,1000,40,33.06,경상남도 창원시 의창구 사림동,1
203,32866635,월세,200,26,29.75,경상남도 창원시 의창구 사림동,1
204,32749391,월세,500,29,29.75,경상남도 창원시 의창구 사림동,1
205,32763950,월세,500,30,33.06,경상남도 창원시 의창구 사림동,1


In [34]:
#max row, max column 설정
pd.options.display.max_columns,pd.options.display.max_rows

(50, 60)

In [35]:
# 함수 만들기
def oneroom(addr):
    # 1.해당 지역의 좌표 값을 얻는다.
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    # 2. 좌표를 geohash로 뱐환 한다.
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # 3. geohash 값으로 매물 ID를 얻는다.
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    items = response.json()['items']
    ids = [items['item_id'] for items in items]
    
    # 4. 매물 ID로 매물 정보를 얻는다
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain" : "zigbang",
        "withCoalition" : "true",
        "item_ids" : ids[:900] 
    }

    response = requests.post(url,params)
    items = response.json()['items']
    
    #5.필요한 컬럼만 추가하여 DataFrame을 만든다.
    columns = ['item_id', 'sales_type','deposit','rent','size_m2','address1','manage_cost']
    
    return pd.DataFrame(items)[columns]
    
    
    
    

In [37]:
addr = "마포구 합정동"
df = oneroom(addr)
df_filtered = df[df['address1'].str.contains(addr)].reset_index(drop=True)
df_filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
95,32794268,월세,1000,90,59.50,서울시 마포구 합정동,2
96,32794303,월세,3000,80,59.50,서울시 마포구 합정동,2
97,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
98,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2


In [43]:
# 모듈 파일 만들기 : .py

UsageError: Line magic function `%%writefile` not found.


In [44]:
%%writefile zigbang.py
import requests
import pandas as pd
import geohash2

def oneroom(addr):
    # 1.해당 지역의 좌표 값을 얻는다.
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    # 2. 좌표를 geohash로 뱐환 한다.
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # 3. geohash 값으로 매물 ID를 얻는다.
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    items = response.json()['items']
    ids = [items['item_id'] for items in items]
    
    # 4. 매물 ID로 매물 정보를 얻는다
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain" : "zigbang",
        "withCoalition" : "true",
        "item_ids" : ids[:900] 
    }

    response = requests.post(url,params)
    items = response.json()['items']
    
    #5.필요한 컬럼만 추가하여 DataFrame을 만든다.
    columns = ['item_id', 'sales_type','deposit','rent','size_m2','address1','manage_cost']
    
    return pd.DataFrame(items)[columns]

Writing zigbang.py


In [45]:
%whos

Variable      Type         Data/Info
------------------------------------
addr          str          마포구 합정동
columns       list         n=7
data          dict         n=13
df            DataFrame          item_id sales_type <...>n\n[744 rows x 7 columns]
df_filtered   DataFrame         item_id sales_type  <...>n\n[100 rows x 7 columns]
geohash       str          wy782
geohash2      module       <module 'geohash2' from '<...>\\geohash2\\__init__.py'>
ids           list         n=207
items         list         n=207
lat           float        35.21626663208008
lng           float        128.6872100830078
oneroom       function     <function oneroom at 0x000001F9848A7820>
params        dict         n=3
pd            module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests      module       <module 'requests' from '<...>\\requests\\__init__.py'>
response      Response     <Response [200]>
url           str          https://apis.zigbang.com/v2/items/list


In [ ]:
%reset